In [ ]:
%env  WORKDIR=/tmp/vault
%env ROLE_NAME=internal-app

Importamos la dirección, localización del fichero de la CA de Vault así como el Vault token usando [python-dotenv](https://pypi.org/project/python-dotenv/). De esta forma evitamos filtrar el root token de Vault

In [2]:
import os
from dotenv import load_dotenv

load_dotenv("/tmp/vault/config.env")

VAULT_TOKEN = os.getenv('VAULT_TOKEN')
VAULT_ADDR = os.getenv('VAULT_ADDR')
VAULT_CACERT = os.getenv('VAULT_CACERT')

Creamos una service account que será usada por un POD en el namespace test

In [ ]:
! kubectl create sa $ROLE_NAME -n test

Extendemos el role creado previamente para permitir el acceso de la SA que se acaba de crear

In [ ]:
%%bash

vault write auth/kubernetes/role/role \
    bound_service_account_names=default,$ROLE_NAME \
    bound_service_account_namespaces=test \
    policies=devk8s \
    ttl=10m

Desplegamos un POD con anotaciones que serán capturadas por el mutating webhook controller. El controller se encargará de modificar el spec del POD

In [20]:
%%bash
cat > $WORKDIR/orgchart.yaml <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: orgchart
  namespace: test
  labels:
    app: orgchart
spec:
  selector:
    matchLabels:
      app: orgchart
  replicas: 1
  template:
    metadata:
      annotations:
        vault.hashicorp.com/agent-inject: 'true'
        vault.hashicorp.com/role: 'role'
        vault.hashicorp.com/agent-inject-secret-database-config.txt: 'database/creds/readonly'
        vault.hashicorp.com/tls-skip-verify: 'true' # Untrusted cert used here
        vault.hashicorp.com/agent-inject-template-database-config.txt: |
          {{- with secret "database/creds/readonly" -}}
          postgresql://{{ .Data.username }}:{{ .Data.password }}@postgres:5432/postgres
          {{- end -}}
      labels:
        app: orgchart
    spec:
      serviceAccountName: $ROLE_NAME
      containers:
        - name: orgchart
          image: nginx
EOF

In [ ]:
! kubectl apply --filename $WORKDIR/orgchart.yaml

### Vault Agents Logs

In [ ]:
%%bash 
kubectl logs -n test\
    $(kubectl get pod -n test -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
    --container vault-agent

### Leemos el secreto del POD

In [ ]:
%%bash
kubectl exec -n test\
    $(kubectl get pod -n test -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
    --container orgchart -- cat /vault/secrets/database-config.txt

Si esperamos dos minutos podemos ver como el secreto ha sido actualizado

In [ ]:
%%bash
sleep 120
kubectl exec -n test\
    $(kubectl get pod -n test -l app=orgchart -o jsonpath="{.items[0].metadata.name}") \
    --container orgchart -- cat /vault/secrets/database-config.txt